In [9]:
import tensorflow as tf
import numpy as np
import keras
import random
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [10]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

testImgGen = test_datagen.flow_from_directory(
    '/content/data',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    interpolation='nearest',
    batch_size = 100, #TODO: CHANGE
    subset="training"
)
valImgGen = test_datagen.flow_from_directory(
    '/content/data',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    interpolation='nearest',
    batch_size = 100, #TODO: CHANGE
    subset="validation"
)


Found 32924 images belonging to 10 classes.
Found 8225 images belonging to 10 classes.


In [11]:
base_model =tf.keras.applications.DenseNet169(
    include_top=False,
    weights='imagenet'
)
for layer in base_model.layers:
  layer.trainable = False
# base_model.summary()
inp = keras.Input(shape=(224, 224, 3), dtype=tf.uint8)
x = tf.cast(inp, tf.float32)
x = tf.keras.applications.densenet.preprocess_input(x)
x = base_model(x)

x = layers.Conv2D(64, (3,3), activation='relu')(x)
x = layers.MaxPool2D(2,2)(x)

# # x = layers.Conv2D(128, (3,3), activation='relu')(x)

x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inp, outputs=x)
model.summary()

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['acc'])

51888128/51877672 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                              

In [12]:

epochs = 1 
histroy = model.fit(
    testImgGen,
    epochs=epochs,
    verbose=1,
    validation_data=valImgGen,
    workers=16
)

330/330 [==============================] - ETA: 0s - loss: 0.2840 - acc: 0.9250 

KeyboardInterrupt: ignored

In [13]:

model.save('/content/saved_model/my_model')


INFO:tensorflow:Assets written to: /content/saved_model/my_model/assets


In [16]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/my_model/ (stored 0%)
  adding: content/saved_model/my_model/variables/ (stored 0%)
  adding: content/saved_model/my_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/saved_model/my_model/variables/variables.index (deflated 78%)
  adding: content/saved_model/my_model/assets/ (stored 0%)
  adding: content/saved_model/my_model/saved_model.pb (deflated 92%)
  adding: content/saved_model/my_model/keras_metadata.pb (deflated 96%)
